In [1]:
import torch
import torch.nn as nn
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score
import pandas as pd
import os
import json
import unicodedata

2023-03-21 12:47:10.622650: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 12:47:25.899945: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-21 12:47:25.900077: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-21 12:47:25.900088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device 

device(type='cuda')

In [4]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [5]:
feature_data = pd.DataFrame()

In [6]:
file_path = []
file_name = []
home_path = '/home/guoy20/Data/json/'
label = []
classification = ['Good','Bad','Inter']
for c in classification:
    path1 = home_path + c 
    filename = os.listdir(path1)
    for name in filename:
        if name != '.ipynb_checkpoints':
            file_path.append(path1 + '/' + name)

            if c == 'Good':
                label.append(2)
            elif c == 'Inter':
                label.append(1)
            elif c == 'Bad':
                label.append(0)
            file_name.append(name)

In [7]:
dataset = []
for p in file_path:
    text = ''
    try:
        with open(p) as f:
            data = json.load(f)
        j = 0
        for item in data:
        
            try:
                text = text + list(item.values())[0]
                
            except:
                text = text
            j += 1
            if j > 5:
                break
    except:
        text = text
    new_str = unicodedata.normalize("NFKD", text)
    dataset.append(new_str)

In [8]:
dataset[0]

'Evaluation Only. Created with Aspose.Words. Copyright 2003-2023 Aspose Pty Ltd.JAMES R. BOCINSKY  OBJECTIVE – FINANCIAL ADVISOR Relationship focused and tenacious professional consistently instilling the utmost  confidence in clients and partners alike. Maximizes people and business potential  by embracing ambiguity and making the complex simple. Unique ability to focus  on details while incorporating big picture strategy and vision.  Constantly exhibits integrity, loyalty, and professionalism in a personable and  polished manner. These attributes have led to proven results throughout the  client acquisition and service cycles. PROFESSIONAL EXPERIENCE '

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [24]:
model_name = "bert-base-uncased"
num_labels = 2 # Number of classes
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [27]:
len(label) == len(dataset)

True

In [28]:
train_texts = dataset
train_labels = label# Class labels for each document

# Encode the training data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)

In [29]:
class DocumentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DocumentDataset(train_encodings, train_labels)

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# Define the Trainer
trainer = Trainer(
    model=model,                         # the PyTorch model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
)

# Train the model
trainer.train()








/home/guoy20/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 130
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:240: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Asser

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
